# Machine Learning for Classification
House price prediction

## Dataset

Dataset is the California Housing Prices from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

Here's a wget-able [link](https://github.com/Ksyula/ML_Engineering/blob/master/02-regression/housing.csv):

```bash
wget https://raw.githubusercontent.com/Ksyula/ML_Engineering/master/02-regression/housing.csv
```

The goal is to create a regression model for predicting housing prices (column `'median_house_value'`).

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score, mean_squared_error
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, Ridge

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

np.__version__, pd.__version__

('1.21.5', '1.4.3')

## Data preparation

In [28]:
data = pd.read_csv('../02-regression/housing.csv')
data.shape

(20640, 10)

In [29]:
variables = ['latitude','longitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income','median_house_value','ocean_proximity']
data = data[variables].fillna(0)
data.head().T

,0,1,2,3,4
latitude,37.88,37.86,37.85,37.85,37.85
longitude,-122.23,-122.22,-122.24,-122.25,-122.25
housing_median_age,41.0,21.0,52.0,52.0,52.0
total_rooms,880.0,7099.0,1467.0,1274.0,1627.0
total_bedrooms,129.0,1106.0,190.0,235.0,280.0
population,322.0,2401.0,496.0,558.0,565.0
households,126.0,1138.0,177.0,219.0,259.0
median_income,8.3252,8.3014,7.2574,5.6431,3.8462
median_house_value,452600.0,358500.0,352100.0,341300.0,342200.0
ocean_proximity,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY


In [30]:
data.dtypes

latitude              float64
longitude             float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object

## Feature engineering

In [31]:
data['rooms_per_household'] = data['total_rooms'] / data['households']
data['bedrooms_per_room'] = data['total_bedrooms'] / data['total_rooms']
data['population_per_household'] = data['population'] / data['households']

### Question 1

What is the most frequent observation (mode) for the column `ocean_proximity`?

Options:
* NEAR BAY
* **<1H OCEAN**
* INLAND
* NEAR OCEAN

In [32]:
data['ocean_proximity'].mode()

0    <1H OCEAN
Name: ocean_proximity, dtype: object

## Set up validation framework

In [33]:
# Split the data in train/val/test sets, with 60%/20%/20% distribution
def set_up_val_framework(val = 0.25, test = 0.2):
    
    df_full_train, df_test = train_test_split(data, test_size = test, random_state=42)
    df_train, df_val = train_test_split(df_full_train, test_size = val, random_state=42)
    
    df_train = df_train.reset_index(drop = True)
    df_val = df_val.reset_index(drop = True)
    df_test = df_test.reset_index(drop = True)
    
    y_train = df_train.median_house_value.values
    y_val = df_val.median_house_value.values
    y_test = df_test.median_house_value.values

    del df_train['median_house_value']
    del df_val['median_house_value']
    del df_test['median_house_value']
    
    return df_train, df_val, df_test, y_train, y_val, y_test

In [34]:
df_train, df_val, df_test, y_train, y_val, y_test = set_up_val_framework()

## EDA

In [35]:
data.isna().sum()

latitude                    0
longitude                   0
housing_median_age          0
total_rooms                 0
total_bedrooms              0
population                  0
households                  0
median_income               0
median_house_value          0
ocean_proximity             0
rooms_per_household         0
bedrooms_per_room           0
population_per_household    0
dtype: int64

### Feature importance: Correlation

#### Question 2

* Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your train dataset.
    - In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
* What are the two features that have the biggest correlation in this dataset?

Options:
* **`total_bedrooms` and `households`**
* `total_bedrooms` and `total_rooms`
* `population` and `households`
* `population_per_household` and `total_rooms`

In [36]:
cor_mat = df_train.corr(method='pearson')
cor_coefs = cor_mat.where(np.triu(np.ones(cor_mat.shape), k=1).astype(bool)).stack().reset_index().rename(columns={0: "coef"})
cor_coefs.sort_values(by = "coef", ascending = False, key=abs).head(3)


,level_0,level_1,coef
35,total_bedrooms,households,0.979399
27,total_rooms,total_bedrooms,0.931546
0,latitude,longitude,-0.925005


### Target variable
Make `median_house_value` binary

* We need to turn the `median_house_value` variable from numeric into binary.
* Let's create a variable `above_average` which is `1` if the `median_house_value` is above its mean value and `0` otherwise.

In [37]:
median_house_value_mean = round(data.median_house_value.mean(), 2)

In [38]:
y_train = [1 if y > median_house_value_mean else 0 for y in y_train]
y_val = [1 if y > median_house_value_mean else 0 for y in y_val]
y_test = [1 if y > median_house_value_mean else 0 for y in y_test]

### Feature importance: Mutual Information
#### Question 3

* Calculate the mutual information score with the (binarized) price for the categorical variable that we have. Use the training set only.
* What is the value of mutual information?
* Round it to 2 decimal digits using `round(score, 2)`

Options:
- 0.263
- 0.00001
- **0.101**
- 0.15555

In [39]:
round(mutual_info_score(df_train.ocean_proximity, y_train), 2)

0.1

## One-hot Encoding

In [40]:
def one_hot_encoding(data):
    
    train_dicts = data.to_dict(orient='records')
    dv = DictVectorizer(sparse = False)
    one_hot_encoded_data = dv.fit_transform(train_dicts)
    return one_hot_encoded_data, dv.get_feature_names_out().tolist()

In [42]:
X_train, feature_names = one_hot_encoding(df_train)
feature_names

['bedrooms_per_room',
 'households',
 'housing_median_age',
 'latitude',
 'longitude',
 'median_income',
 'ocean_proximity=<1H OCEAN',
 'ocean_proximity=INLAND',
 'ocean_proximity=ISLAND',
 'ocean_proximity=NEAR BAY',
 'ocean_proximity=NEAR OCEAN',
 'population',
 'population_per_household',
 'rooms_per_household',
 'total_bedrooms',
 'total_rooms']

In [44]:
X_val = one_hot_encoding(df_val)[0]
X_test = one_hot_encoding(df_test)[0]

## Logistic Regression
### Question 4

* Now let's train a logistic regression
* Remember that we have one categorical variable `ocean_proximity` in the data. Include it using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

Options:
- 0.60
- 0.72
- **0.84**
- 0.95

In [18]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [19]:
model.intercept_[0]

-0.08087359748906243

In [20]:
model.coef_[0].round(3)

array([ 0.171,  0.004,  0.036,  0.116,  0.087,  1.209,  0.471, -1.702,
        0.018,  0.295,  0.838, -0.002,  0.01 , -0.014,  0.002, -0.   ])

In [21]:
y_pred = model.predict(X_val)
global_acc = (y_val == y_pred).mean()
global_acc.round(2)

0.84

## Feature elimination
### Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 
* Which of following feature has the smallest difference? 
   * `total_rooms`
   * **`total_bedrooms`**
   * `population`
   * `households`

> **note**: the difference doesn't have to be positive

In [22]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'bedrooms_per_room': 0.171,
 'households': 0.004,
 'housing_median_age': 0.036,
 'latitude': 0.116,
 'longitude': 0.087,
 'median_income': 1.209,
 'ocean_proximity=<1H OCEAN': 0.471,
 'ocean_proximity=INLAND': -1.702,
 'ocean_proximity=ISLAND': 0.018,
 'ocean_proximity=NEAR BAY': 0.295,
 'ocean_proximity=NEAR OCEAN': 0.838,
 'population': -0.002,
 'population_per_household': 0.01,
 'rooms_per_household': -0.014,
 'total_bedrooms': 0.002,
 'total_rooms': -0.0}

In [46]:
for f in ['total_rooms', 'total_bedrooms', 'population', 'households']:
    features = df_train.columns.tolist()
    features.remove(f)
    # Prepare
    x_train = one_hot_encoding(df_train[features])[0]
    # Train                    
    model = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    model.fit(x_train, y_train)
    # Validate
    x_val = one_hot_encoding(df_val[features])[0]

    y_pred = model.predict(x_val)
    acc = (y_val == y_pred).mean()
    print(abs(global_acc - acc))

0.0014534883720930258
0.00024224806201555982
0.009689922480620172
0.002664728682170492


### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn
* We'll need to use the original column `'median_house_value'`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model (`model = Ridge(alpha=a, solver="sag", random_state=42)`) on the training data.
* This model has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`
* Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.

If there are multiple options, select the smallest `alpha`.

Options:
- **0**
- 0.01
- 0.1
- 1
- 10

In [56]:
df_train, df_val, df_test, y_train, y_val, y_test = set_up_val_framework()

In [57]:
y_train = np.log(y_train)
y_val = np.log(y_val)
y_test = np.log(y_test)

In [58]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [62]:
for a in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=a, solver="sag", random_state=42)
    
    x_train = one_hot_encoding(df_train)[0]
    
    model.fit(x_train, y_train)
    
    x_val = one_hot_encoding(df_val)[0]
    
    y_pred = model.predict(x_val)
    
    print(rmse(y_val, y_pred).round(3))

0.524
0.524
0.524
0.524
0.524
